In [1]:
import requests
import re, json
from bs4 import BeautifulSoup

In [92]:
url = 'https://news.abs-cbn.com/life/'

link = 'https://news.abs-cbn.com'
cont = True

failing = [
    re.compile(r'\A/life/0[1-2]/../21.*'),
    re.compile(r'\A/life/03/(0.)|(10)/21.*'),
    re.compile(r'\A/life/../../((20)|([0-1|3-9].)).*')
]


In [4]:
news = []

while (cont): 
    page=requests.get(url)   
    soup = BeautifulSoup(page.content, 'html.parser')
    
    articles = soup.find_all(href=re.compile(r'\A/life/03/1[1|2]/21.*'))
    
    for i in articles:
        if (i.parent.name == 'article'):

            article_page = requests.get(link + i['href'])
            article_soup = BeautifulSoup(article_page.content, 'html.parser')

            author = article_soup.find('meta', {'name': 'author'})['content']
            title = article_soup.find('meta', {'property': 'og:title'})['content']
            date = article_soup.find(class_='date-posted').text

            article_element = article_soup.find(class_='article-content').find_all(['p', 'img', 'iframe'])

            full_article = ''

            for i in article_element:
                if (i.has_attr('src')):               # keeps embedded images and videos
                    full_article += i['src'] + '\n'
                else:
                    full_article += i.text + '\n'

            news.append({
                'date': date,
                'title': title,
                'full_article': full_article,
                'author': author
            })
    
    fail = soup.find_all(href=failing) 
    cont = fail == []
    
    url = link + soup.find(title="Next")['href']

In [8]:
with open('News.json', 'w') as outfile:
    json.dump(news, outfile)